In [1]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata, skew, kurtosis
from scipy.optimize import minimize
import yfinance as yf
from pandas.tseries.offsets import MonthEnd, WeekOfMonth
import warnings
warnings.filterwarnings("ignore")

## Get historical S&P500 constituents data

In [2]:
sp500 = pd.read_csv('sp500.csv')
sp500_hist = pd.read_csv('sp500_historical.csv')
# csv files credit to https://github.com/fja05680/sp500

In [3]:
start_date = '2015-12-30'
sp500_hist['date'] = pd.to_datetime(sp500_hist['date'])
sp500_hist = sp500_hist.loc[sp500_hist['date'] >= start_date].reset_index(drop=True)
sp500_hist

,date,tickers
0,2015-12-30,"A,AABA,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,..."
1,2015-12-31,"A,AABA,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,..."
2,2016-01-04,"A,AABA,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,..."
3,2016-01-05,"A,AABA,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,..."
4,2016-01-06,"A,AABA,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,..."
...,...,...
437,2023-05-04,"A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,ADI,..."
438,2023-05-16,"A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,ADI,..."
439,2023-06-07,"A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,ADI,..."
440,2023-06-10,"A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,ADI,..."


In [4]:
years = range(2016, 2024)
months = [3, 6, 9, 12]
first_rebalaced = []

# calculate the Monday after the third Friday
for year in years:
    for month in months:
        # find the date of the third Friday in the given month and year
        third_friday = pd.date_range(start=f'{year}-{month}-01', periods=1, freq=WeekOfMonth(week=2, weekday=4))[0]
        # calculate the following Monday
        following_monday = third_friday + pd.DateOffset(days=3)
        first_rebalaced.append(following_monday)

first_rebalaced_str = [date.strftime('%Y-%m-%d') for date in first_rebalaced]
first_rebalaced_str

['2016-03-21',
 '2016-06-20',
 '2016-09-19',
 '2016-12-19',
 '2017-03-20',
 '2017-06-19',
 '2017-09-18',
 '2017-12-18',
 '2018-03-19',
 '2018-06-18',
 '2018-09-24',
 '2018-12-24',
 '2019-03-18',
 '2019-06-24',
 '2019-09-23',
 '2019-12-23',
 '2020-03-23',
 '2020-06-22',
 '2020-09-21',
 '2020-12-21',
 '2021-03-22',
 '2021-06-21',
 '2021-09-20',
 '2021-12-20',
 '2022-03-21',
 '2022-06-20',
 '2022-09-19',
 '2022-12-19',
 '2023-03-20',
 '2023-06-19',
 '2023-09-18',
 '2023-12-18']

In [5]:
tickers_dict = {}
for date in first_rebalaced:
    mask = sp500_hist['date'] == date
    if mask.any():  # check if there is at least one match
        idx = mask.idxmax()
        tickers_dict[date.strftime('%Y-%m-%d')] = sp500_hist.at[idx, 'tickers']
    else:
        # if the date is not found, find the next closest date in the dataframe
        next_date_mask = sp500_hist['date'] > date
        if next_date_mask.any():
            next_date_idx = next_date_mask.idxmax()
            tickers_dict[date.strftime('%Y-%m-%d')] = sp500_hist.at[next_date_idx, 'tickers']
        else:
            # in case there are no future dates in the dataframe
            tickers_dict[date.strftime('%Y-%m-%d')] = []
        
tickers_dict

{'2016-03-21': 'A,AABA,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,ADS,ADSK,ADT,AEE,AEP,AES,AET,AFL,AGN,AIG,AIV,AIZ,AKAM,ALL,ALLE,ALXN,AMAT,AME,AMG,AMGN,AMP,AMT,AMZN,AN,ANDV,ANTM,AON,APA,APC,APD,APH,APTV,ARG,ARNC,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBBY,BBT,BBY,BCR,BDX,BEN,BF.B,BHGE,BIIB,BK,BKNG,BLK,BLL,BMY,BRK.B,BSX,BWA,BXLT,BXP,C,CA,CAG,CAH,CAM,CAT,CB,CBRE,CBS,CCE,CCEP,CCI,CCL,CELG,CERN,CF,CFG,CHD,CHK,CHRW,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNP,COF,COG,COL,COP,COST,CPB,CPGX,CPRI,CRM,CSCO,CSRA,CSX,CTAS,CTL,CTSH,CTXS,CVC,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DLTR,DNB,DO,DOV,DOW,DRI,DTE,DUK,DVA,DVN,EA,EBAY,ECL,ED,EFX,EIX,EL,EMC,EMN,EMR,ENDP,EOG,EQIX,EQR,EQT,ES,ESRX,ESS,ESV,ETFC,ETN,ETR,EW,EXC,EXPD,EXPE,EXR,F,FAST,FB,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLR,FLS,FMC,FOX,FOXA,FRT,FSLR,FTI,FTR,GAS,GD,GE,GGP,GILD,GIS,GLW,GM,GME,GOOG,GOOGL,GPC,GPS,GRMN,GS,GT,GWW,HAL,HAR,HAS,HBAN,HBI,HCA,HCP,HD,HES,HIG,HOG,HON,HOT,HP,HPE,HPQ,HRB,HRL,HRS,HSIC,HST,HSY,HUM,IBM,ICE

In [6]:
tickers_dict['2023-09-18'] = sp500['Symbol'].tolist()
tickers_dict

{'2016-03-21': 'A,AABA,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,ADS,ADSK,ADT,AEE,AEP,AES,AET,AFL,AGN,AIG,AIV,AIZ,AKAM,ALL,ALLE,ALXN,AMAT,AME,AMG,AMGN,AMP,AMT,AMZN,AN,ANDV,ANTM,AON,APA,APC,APD,APH,APTV,ARG,ARNC,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBBY,BBT,BBY,BCR,BDX,BEN,BF.B,BHGE,BIIB,BK,BKNG,BLK,BLL,BMY,BRK.B,BSX,BWA,BXLT,BXP,C,CA,CAG,CAH,CAM,CAT,CB,CBRE,CBS,CCE,CCEP,CCI,CCL,CELG,CERN,CF,CFG,CHD,CHK,CHRW,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNP,COF,COG,COL,COP,COST,CPB,CPGX,CPRI,CRM,CSCO,CSRA,CSX,CTAS,CTL,CTSH,CTXS,CVC,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DLTR,DNB,DO,DOV,DOW,DRI,DTE,DUK,DVA,DVN,EA,EBAY,ECL,ED,EFX,EIX,EL,EMC,EMN,EMR,ENDP,EOG,EQIX,EQR,EQT,ES,ESRX,ESS,ESV,ETFC,ETN,ETR,EW,EXC,EXPD,EXPE,EXR,F,FAST,FB,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLR,FLS,FMC,FOX,FOXA,FRT,FSLR,FTI,FTR,GAS,GD,GE,GGP,GILD,GIS,GLW,GM,GME,GOOG,GOOGL,GPC,GPS,GRMN,GS,GT,GWW,HAL,HAR,HAS,HBAN,HBI,HCA,HCP,HD,HES,HIG,HOG,HON,HOT,HP,HPE,HPQ,HRB,HRL,HRS,HSIC,HST,HSY,HUM,IBM,ICE

In [7]:
# Adjusting the dictionary to organize the ticker values as specified below:
# 1. Replace commas with spaces in string values.
# 2. Convert list values to a single string with tickers separated by spaces.
# 3. Delete entries with empty lists as values.

tickers_dict_adjusted = {}
for date, tickers in tickers_dict.items():
    if tickers is None:
        continue  # skip None values (no future dates available)
    if isinstance(tickers, list):
        # convert list to a single string with tickers separated by spaces
        tickers_str = ' '.join(tickers)
        # skip adding to the dictionary if the list is empty
        if tickers_str:
            tickers_dict_adjusted[date] = tickers_str
    elif isinstance(tickers, str):
        tickers_str = tickers.replace(',', ' ')
        tickers_dict_adjusted[date] = tickers_str

tickers_dict_adjusted

{'2016-03-21': 'A AABA AAL AAP AAPL ABBV ABC ABT ACN ADBE ADI ADM ADP ADS ADSK ADT AEE AEP AES AET AFL AGN AIG AIV AIZ AKAM ALL ALLE ALXN AMAT AME AMG AMGN AMP AMT AMZN AN ANDV ANTM AON APA APC APD APH APTV ARG ARNC ATVI AVB AVGO AVY AWK AXP AZO BA BAC BAX BBBY BBT BBY BCR BDX BEN BF.B BHGE BIIB BK BKNG BLK BLL BMY BRK.B BSX BWA BXLT BXP C CA CAG CAH CAM CAT CB CBRE CBS CCE CCEP CCI CCL CELG CERN CF CFG CHD CHK CHRW CI CINF CL CLX CMA CMCSA CME CMG CMI CMS CNP COF COG COL COP COST CPB CPGX CPRI CRM CSCO CSRA CSX CTAS CTL CTSH CTXS CVC CVS CVX CXO D DAL DD DE DFS DG DGX DHI DHR DIS DISCA DISCK DLTR DNB DO DOV DOW DRI DTE DUK DVA DVN EA EBAY ECL ED EFX EIX EL EMC EMN EMR ENDP EOG EQIX EQR EQT ES ESRX ESS ESV ETFC ETN ETR EW EXC EXPD EXPE EXR F FAST FB FCX FDX FE FFIV FIS FISV FITB FLIR FLR FLS FMC FOX FOXA FRT FSLR FTI FTR GAS GD GE GGP GILD GIS GLW GM GME GOOG GOOGL GPC GPS GRMN GS GT GWW HAL HAR HAS HBAN HBI HCA HCP HD HES HIG HOG HON HOT HP HPE HPQ HRB HRL HRS HSIC HST HSY HUM IBM ICE

In [8]:
sorted_dates = sorted(tickers_dict_adjusted.keys(), key=lambda x: pd.to_datetime(x))
tickers_with_previous = {}

# add tickers from the previous date to the current one, avoiding duplicates
for i, date in enumerate(sorted_dates):
    current_set = set(tickers_dict_adjusted[date].split())
    if i > 0:
        # get the previous date's tickers and combine them with the current set
        previous_date = sorted_dates[i-1]
        previous_set = set(tickers_dict_adjusted[previous_date].split())
        current_set = current_set.union(previous_set)
    tickers_with_previous[date] = ' '.join(sorted(current_set))

tickers_with_previous

{'2016-03-21': 'A AABA AAL AAP AAPL ABBV ABC ABT ACN ADBE ADI ADM ADP ADS ADSK ADT AEE AEP AES AET AFL AGN AIG AIV AIZ AKAM ALL ALLE ALXN AMAT AME AMG AMGN AMP AMT AMZN AN ANDV ANTM AON APA APC APD APH APTV ARG ARNC ATVI AVB AVGO AVY AWK AXP AZO BA BAC BAX BBBY BBT BBY BCR BDX BEN BF.B BHGE BIIB BK BKNG BLK BLL BMY BRK.B BSX BWA BXLT BXP C CA CAG CAH CAM CAT CB CBRE CBS CCE CCEP CCI CCL CELG CERN CF CFG CHD CHK CHRW CI CINF CL CLX CMA CMCSA CME CMG CMI CMS CNP COF COG COL COP COST CPB CPGX CPRI CRM CSCO CSRA CSX CTAS CTL CTSH CTXS CVC CVS CVX CXO D DAL DD DE DFS DG DGX DHI DHR DIS DISCA DISCK DLTR DNB DO DOV DOW DRI DTE DUK DVA DVN EA EBAY ECL ED EFX EIX EL EMC EMN EMR ENDP EOG EQIX EQR EQT ES ESRX ESS ESV ETFC ETN ETR EW EXC EXPD EXPE EXR F FAST FB FCX FDX FE FFIV FIS FISV FITB FLIR FLR FLS FMC FOX FOXA FRT FSLR FTI FTR GAS GD GE GGP GILD GIS GLW GM GME GOOG GOOGL GPC GPS GRMN GS GT GWW HAL HAR HAS HBAN HBI HCA HCP HD HES HIG HOG HON HOT HP HPE HPQ HRB HRL HRS HSIC HST HSY HUM IBM ICE

In [11]:
# adjust the end date to be the day before the next key date
def adjust_end_date(date_list, current_index):
    if current_index < len(date_list) - 1:
        return pd.to_datetime(date_list[current_index + 1]) - pd.Timedelta(days=1)
    return pd.to_datetime('today')

dates = list(tickers_with_previous.keys())
concatenated_data = pd.DataFrame()

for index, (start_date, tickers) in enumerate(tickers_dict_adjusted.items()):
    end_date = adjust_end_date(dates, index)
    data = yf.download(tickers, start=start_date, end=end_date)
    concatenated_data = pd.concat([concatenated_data, data])

concatenated_data.head()

[*********************100%***********************]  504 of 504 completed

91 Failed downloads:
- LVLT: No data found for this date range, symbol may be delisted
- ETFC: No timezone found, symbol may be delisted
- MON: No timezone found, symbol may be delisted
- MJN: No data found for this date range, symbol may be delisted
- XEC: No timezone found, symbol may be delisted
- BBT: No timezone found, symbol may be delisted
- FLIR: No timezone found, symbol may be delisted
- WYND: No timezone found, symbol may be delisted
- SNDK: No data found for this date range, symbol may be delisted
- RHT: No timezone found, symbol may be delisted
- XLNX: No timezone found, symbol may be delisted
- SPLS: No data found for this date range, symbol may be delisted
- TWC: No data found for this date range, symbol may be delisted
- MNK: No timezone found, symbol may be delisted
- ADS: No timezone found, symbol may be delisted
- CBS: No timezone found, symbol may be delisted
- DNB: Data doesn't exist for star

[*********************100%***********************]  506 of 506 completed

79 Failed downloads:
- MON: No timezone found, symbol may be delisted
- ETFC: No timezone found, symbol may be delisted
- MJN: No data found for this date range, symbol may be delisted
- LVLT: No data found for this date range, symbol may be delisted
- XEC: No timezone found, symbol may be delisted
- BBT: No timezone found, symbol may be delisted
- WYND: No timezone found, symbol may be delisted
- FLIR: No timezone found, symbol may be delisted
- RHT: No timezone found, symbol may be delisted
- XLNX: No timezone found, symbol may be delisted
- SPLS: No data found for this date range, symbol may be delisted
- MNK: No timezone found, symbol may be delisted
- ADS: No timezone found, symbol may be delisted
- CBS: No timezone found, symbol may be delisted
- DNB: Data doesn't exist for startDate = 1474257600, endDate = 1482037200
- LB: No timezone found, symbol may be delisted
- BLL: No timezone found, symbol may be de

[*********************100%***********************]  506 of 506 completed

70 Failed downloads:
- MON: No timezone found, symbol may be delisted
- ETFC: No timezone found, symbol may be delisted
- LVLT: No data found for this date range, symbol may be delisted
- MJN: No data found for this date range, symbol may be delisted
- XEC: No timezone found, symbol may be delisted
- BBT: No timezone found, symbol may be delisted
- FLIR: No timezone found, symbol may be delisted
- WYND: No timezone found, symbol may be delisted
- RHT: No timezone found, symbol may be delisted
- XLNX: No timezone found, symbol may be delisted
- SPLS: No data found for this date range, symbol may be delisted
- MNK: No timezone found, symbol may be delisted
- ADS: No timezone found, symbol may be delisted
- CBS: No timezone found, symbol may be delisted
- DNB: Data doesn't exist for startDate = 1489982400, endDate = 1497758400
- BLL: No timezone found, symbol may be delisted
- LB: No timezone found, symbol may be de

[*********************100%***********************]  506 of 506 completed

64 Failed downloads:
- ETFC: No timezone found, symbol may be delisted
- XEC: No timezone found, symbol may be delisted
- MON: No timezone found, symbol may be delisted
- BBT: No timezone found, symbol may be delisted
- FLIR: No timezone found, symbol may be delisted
- WYND: No timezone found, symbol may be delisted
- RHT: No timezone found, symbol may be delisted
- XLNX: No timezone found, symbol may be delisted
- DRE: No timezone found, symbol may be delisted
- ADS: No timezone found, symbol may be delisted
- CBS: No timezone found, symbol may be delisted
- LB: No timezone found, symbol may be delisted
- BLL: No timezone found, symbol may be delisted
- AGN: No timezone found, symbol may be delisted
- DISCA: No timezone found, symbol may be delisted
- PBCT: No timezone found, symbol may be delisted
- ARNC: No timezone found, symbol may be delisted
- WLTW: No timezone found, symbol may be delisted
- FB: No timezo

[*********************100%***********************]  505 of 505 completed

61 Failed downloads:
- XEC: No timezone found, symbol may be delisted
- ETFC: No timezone found, symbol may be delisted
- BBT: No timezone found, symbol may be delisted
- FLIR: No timezone found, symbol may be delisted
- RHT: No timezone found, symbol may be delisted
- XLNX: No timezone found, symbol may be delisted
- DRE: No timezone found, symbol may be delisted
- TWTR: No timezone found, symbol may be delisted
- ADS: No timezone found, symbol may be delisted
- CBS: No timezone found, symbol may be delisted
- BLL: No timezone found, symbol may be delisted
- LB: No timezone found, symbol may be delisted
- AGN: No timezone found, symbol may be delisted
- DISCA: No timezone found, symbol may be delisted
- PBCT: No timezone found, symbol may be delisted
- WLTW: No timezone found, symbol may be delisted
- ARNC: No timezone found, symbol may be delisted
- FB: No timezone found, symbol may be delisted
- DWDP: No timez

[*********************100%***********************]  505 of 505 completed

57 Failed downloads:
- ETFC: No timezone found, symbol may be delisted
- XEC: No timezone found, symbol may be delisted
- BBT: No timezone found, symbol may be delisted
- FLIR: No timezone found, symbol may be delisted
- RHT: No timezone found, symbol may be delisted
- XLNX: No timezone found, symbol may be delisted
- DRE: No timezone found, symbol may be delisted
- TWTR: No timezone found, symbol may be delisted
- ADS: No timezone found, symbol may be delisted
- CBS: No timezone found, symbol may be delisted
- LB: No timezone found, symbol may be delisted
- BLL: No timezone found, symbol may be delisted
- AGN: No timezone found, symbol may be delisted
- DISCA: No timezone found, symbol may be delisted
- PBCT: No timezone found, symbol may be delisted
- ARNC: No timezone found, symbol may be delisted
- WLTW: No timezone found, symbol may be delisted
- FB: No timezone found, symbol may be delisted
- MYL: No timezo

[*********************100%***********************]  505 of 505 completed

39 Failed downloads:
- ETFC: No timezone found, symbol may be delisted
- FLIR: No timezone found, symbol may be delisted
- XLNX: No timezone found, symbol may be delisted
- DRE: No timezone found, symbol may be delisted
- TWTR: No timezone found, symbol may be delisted
- LB: No timezone found, symbol may be delisted
- BLL: No timezone found, symbol may be delisted
- DISCA: No timezone found, symbol may be delisted
- PBCT: No timezone found, symbol may be delisted
- WLTW: No timezone found, symbol may be delisted
- FB: No timezone found, symbol may be delisted
- MYL: No timezone found, symbol may be delisted
- MXIM: No timezone found, symbol may be delisted
- RE: No timezone found, symbol may be delisted
- BRK.B: No timezone found, symbol may be delisted
- VAR: No timezone found, symbol may be delisted
- COG: No timezone found, symbol may be delisted
- DISCK: No timezone found, symbol may be delisted
- CXO: No tim

[*********************100%***********************]  505 of 505 completed

27 Failed downloads:
- XLNX: No timezone found, symbol may be delisted
- DRE: No timezone found, symbol may be delisted
- TWTR: No timezone found, symbol may be delisted
- BLL: No timezone found, symbol may be delisted
- DISCA: No timezone found, symbol may be delisted
- PBCT: No timezone found, symbol may be delisted
- WLTW: No timezone found, symbol may be delisted
- FB: No timezone found, symbol may be delisted
- RE: No timezone found, symbol may be delisted
- BRK.B: No timezone found, symbol may be delisted
- COG: No timezone found, symbol may be delisted
- DISCK: No timezone found, symbol may be delisted
- ABC: No timezone found, symbol may be delisted
- ANTM: No timezone found, symbol may be delisted
- SIVB: No timezone found, symbol may be delisted
- CTXS: No timezone found, symbol may be delisted
- CERN: No timezone found, symbol may be delisted
- INFO: No timezone found, symbol may be delisted
- BF.B: No

Adj Close                                                        \
                    A AABA        AAL         AAP       AAPL       ABBV ABC   
Date                                                                          
2016-03-21  37.676010  NaN  41.655258  153.697693  24.585201  42.236679 NaN   
2016-03-22  37.894566  NaN  40.974892  153.290726  24.773224  43.360275 NaN   
2016-03-23  37.504971  NaN  40.169971  156.100937  24.636267  42.500610 NaN   
2016-03-24  37.466965  NaN  38.809242  156.808319  24.529480  42.319622 NaN   
2016-03-28  37.552490  NaN  39.202126  156.750122  24.418060  42.312080 NaN   

                                              ... Open                    \
                  ABT         ACN       ADBE  ... AXON  EG  FI PANW RVTY   
Date                                          ...                          
2016-03-21  36.491714   97.750748  92.500000  ...  NaN NaN NaN  NaN  NaN   
2016-03-22  36.733429   97.452492  92.559998  ...  NaN NaN NaN  NaN  NaN   
2016-03-23  36.509617   97.298874  92.160004  ...  NaN NaN NaN  NaN  NaN   
2016-03-24  36.482754  103.299828  92.519997  ...  NaN NaN NaN  NaN  NaN   
2016-03-28  36.599140  103.182335  92.400002  ...  NaN NaN NaN  NaN  NaN   

           Volume                    
             AXON  EG  FI PANW RVTY  
Date                                 
2016-03-21    NaN NaN NaN  NaN  NaN  
2016-03-22    NaN NaN NaN  NaN  NaN  
2016-03-23    NaN NaN NaN  NaN  NaN  
2016-03-24    NaN NaN NaN  NaN  NaN  
2016-03-28    NaN NaN NaN  NaN  NaN  

[5 rows x 4032 columns]

In [12]:
# concatenated_data.to_csv("sp500_rebalanced_v2.csv")

In [15]:
df_adjclose = concatenated_data.xs('Adj Close', level=0, axis=1)
df_close = concatenated_data.xs('Close', level=0, axis=1)
df_open = concatenated_data.xs('High', level=0, axis=1)
df_high = concatenated_data.xs('Low', level=0, axis=1)
df_low = concatenated_data.xs('High', level=0, axis=1)
df_volume = concatenated_data.xs('Volume', level=0, axis=1)
nan = df_adjclose.columns[df_adjclose.isna().all()].tolist()

In [16]:
cols_to_drop = [col for col in concatenated_data.columns if col[1] in nan]
rebalanced_v3 = concatenated_data.drop(cols_to_drop, axis=1)
rebalanced_v3

Adj Close                                                 \
                     A        AAL         AAP        AAPL        ABBV   
Date                                                                    
2016-03-21   37.676010  41.655258  153.697693   24.585201   42.236679   
2016-03-22   37.894566  40.974892  153.290726   24.773224   43.360275   
2016-03-23   37.504971  40.169971  156.100937   24.636267   42.500610   
2016-03-24   37.466965  38.809242  156.808319   24.529480   42.319622   
2016-03-28   37.552490  39.202126  156.750122   24.418060   42.312080   
...                ...        ...         ...         ...         ...   
2023-12-04  128.880005  13.350000   55.630001  189.429993  144.149994   
2023-12-05  127.879997  13.130000   53.160000  193.419998  144.570007   
2023-12-06  128.910004  13.480000   55.160000  192.320007  146.380005   
2023-12-07  128.679993  13.910000   56.250000  194.270004  147.970001   
2023-12-08  127.199997  13.760000   56.270000  195.710007  149.279999   

                                                                       ...  \
                   ABT         ACN        ADBE         ADI        ADM  ...   
Date                                                                   ...   
2016-03-21   36.491714   97.750748   92.500000   52.684582  31.911703  ...   
2016-03-22   36.733429   97.452492   92.559998   52.389156  31.443523  ...   
2016-03-23   36.509617   97.298874   92.160004   52.219055  30.599119  ...   
2016-03-24   36.482754  103.299828   92.519997   52.496586  30.490437  ...   
2016-03-28   36.599140  103.182335   92.400002   52.210106  30.390106  ...   
...                ...         ...         ...         ...        ...  ...   
2023-12-04  105.190002  336.429993  604.559998  183.259995  74.629997  ...   
2023-12-05  104.349998  335.829987  602.219971  180.630005  74.139999  ...   
2023-12-06  104.940002  335.410004  595.700012  180.570007  73.190002  ...   
2023-12-07  104.050003  335.100006  608.780029  184.380005  73.830002  ...   
2023-12-08  104.510002  337.230011  610.010010  184.880005  73.750000  ...   

                  Open                                                 \
                  AXON          EG          FI        PANW       RVTY   
Date                                                                    
2016-03-21         NaN         NaN         NaN         NaN        NaN   
2016-03-22         NaN         NaN         NaN         NaN        NaN   
2016-03-23         NaN         NaN         NaN         NaN        NaN   
2016-03-24         NaN         NaN         NaN         NaN        NaN   
2016-03-28         NaN         NaN         NaN         NaN        NaN   
...                ...         ...         ...         ...        ...   
2023-12-04  229.600006  405.679993  130.110001  294.239990  90.570000   
2023-12-05  234.020004  397.989990  130.759995  282.540009  91.150002   
2023-12-06  232.639999  397.200012  131.000000  293.799988  89.989998   
2023-12-07  233.330002  390.510010  131.500000  288.179993  90.669998   
2023-12-08  235.410004  384.190002  131.500000  290.640015  91.430000   

              Volume                                            
                AXON        EG         FI       PANW      RVTY  
Date                                                            
2016-03-21       NaN       NaN        NaN        NaN       NaN  
2016-03-22       NaN       NaN        NaN        NaN       NaN  
2016-03-23       NaN       NaN        NaN        NaN       NaN  
2016-03-24       NaN       NaN        NaN        NaN       NaN  
2016-03-28       NaN       NaN        NaN        NaN       NaN  
...              ...       ...        ...        ...       ...  
2023-12-04  526600.0  517400.0  2466600.0  4642100.0  515300.0  
2023-12-05  387900.0  333500.0  2291900.0  3200600.0  483400.0  
2023-12-06  267000.0  452000.0  2613500.0  3067700.0  505000.0  
2023-12-07  314300.0  332200.0  2634400.0  4306300.0  666200.0  
2023-12-08  281800.0  292500.

In [17]:
# no data for some tickers - for example, no data under "FB"
len(df_adjclose.columns[df_adjclose.isna().all()])

100

Ticker symbol changes can occur due to various reasons such as mergers, acquisitions, rebrandings, corporate restructurings, or even delistings.
- AABA: Shutdown
- ABC: COR - not listed
- ADS: ADS/ADS.DE not found
- ...

Excluding all would introduce:
- survivorship bias/look-ahead bias/selection bias
- exclusion of poor performers and some top performers
- risk underestimation (maybe this is the only legit cons though)

In [18]:
# df_adjclose.drop(columns=nan, inplace=True)
# df_close.drop(columns=nan, inplace=True)
# df_open.drop(columns=nan, inplace=True)
# df_high.drop(columns=nan, inplace=True)
# df_low.drop(columns=nan, inplace=True)
# df_volume.drop(columns=nan, inplace=True)

## Cross-sectional processors

In [19]:
# sector data is from sp500.csv, but since it only includes the current S&P500 constituents, the rest (historical)
# constituents are label by hands
updated_sector = pd.read_csv('sector.csv')
updated_sector.set_index('ticker', inplace=True)
sector = updated_sector['sector']
sector

ticker
A                  Health Care
AAL                Industrials
AAP     Consumer Discretionary
AAPL    Information Technology
ABBV               Health Care
                 ...          
AXON               Industrials
EG                  Financials
FI                  Financials
PANW    Information Technology
RVTY               Health Care
Name: sector, Length: 574, dtype: object

In [20]:
def group_neutralize(df, group=sector):
    """
    type of 'group': <class 'pandas.core.series.Series'>
    return the demeaned within each group
    """
    return df.sub(df.groupby(group, axis=1).transform('mean'), axis=1)

def group_standardize(df, group=sector):
    return df.sub(df.groupby(group, axis=1).transform('mean'), axis=1).div(df.groupby(group, axis=1).transform('std'), axis=1)

def group_rank(df, group=sector):
    return df.groupby(group, axis=1).rank(axis=0)

def group_scale(df, group=sector):
    """
    normalization scaling
    rather sensitive to outliers and does not retain the original distribution
    """
    group_min = df.groupby(group, axis=1).transform('min')
    group_max = df.groupby(group, axis=1).transform('max')
    return (df - group_min).div(group_max - group_min)

def demean(df):
    return df.sub(df.mean(axis=1), axis=0)

def standardize(df):
    return df.sub(df.mean(axis=1), axis=0).div(df.std(axis=1), axis=0)

def rank(df):
    return df.rank(axis=1, method='dense')

def scale(df):
    return (df.sub(df.min(axis=1), axis=0)).div(df.max(axis=1) - df.min(axis=1), axis=0)

## Time-series processors

In [21]:
def returns(df,log=True):
    if log==True:
        return df.rolling(2).apply(lambda x: np.log(x.iloc[-1] / x.iloc[0]))
    return df.rolling(2).apply(lambda x: x.iloc[-1] / x.iloc[0]) - 1

def tssum(df, window=10):
    return df.rolling(window).sum()

def tsma(df, window=10):
    return df.rolling(window).mean()

def tsstddev(df, window=10):
    return df.rolling(window).std()

def tscorrelation(x, y, window=10):
    return x.rolling(window).corr(y).fillna(0).replace([np.inf, -np.inf], 0)

def tscovariance(x, y, window=10):
    return x.rolling(window).cov(y)

def rolling_rank(na):
    # auxiliary function to be used in 'tsrank'
    return rankdata(na, method='dense')[-1]

def tsrank(df, window=10):
    return df.rolling(window).apply(rolling_rank)

def delta(df, period=1):
    return df.diff(period)

def delay(df, period=1):
    # return a lag
    return df.shift(period)

def tsargmax(df, window=10):
    # return the index+1 of the max value in the time series in past N periods
    return df.rolling(window).apply(np.argmax) + 1 

def ts_argmin(df, window=10):
    # return the index+1 of the min value in the time series in past N periods
    return df.rolling(window).apply(np.argmin) + 1

def decay_linear(df, period=10):
    # return the linear decay on input for the past N periods
    weights = np.array(range(1, period+1))
    sum_weights = np.sum(weights)
    return df.rolling(period).apply(lambda x: np.sum(weights*x) / sum_weights)

def tsskew(df, window=5):
    # define a wrapper function for skew with nan_policy='omit'
    def skew_with_nan_policy(x):
        return skew(x, nan_policy='omit')
    rolling_skewness = df.rolling(window=window).apply(skew_with_nan_policy)
    return rolling_skewness

def tskurt(df, window=5):
    # define a wrapper function for skew with nan_policy='omit'
    def kurt_with_nan_policy(x):
        return kurtosis(x, nan_policy='omit')
    rolling_kurtosis = df.rolling(window=window).apply(kurt_with_nan_policy)
    return rolling_kurtosis

## Other processors
- risk neutralization
- ...

In [22]:
def scale_scores(df, top_percentile=0.5, bottom_percentile=0.5, ls_total=200000, asset_allocation=0.5):
    """
    top/bottom percentile: there would be exposure on only the top percentile values and the bottom percentile values of the stocks
    asset allocation: percent of capital allocated to the long side
    'asset_allocation=0.5' meaning abs(long amount) = abs(short amount) = 1/2*ls_total
    return the scaled alpha values
    """
    
    df = demean(df)
    # calculate the number of stocks to include on each side
    num_stocks = df.shape[1]
    num_long = int(num_stocks * top_percentile)
    num_short = int(num_stocks * bottom_percentile)
    
    # initialize scaled performance df
    scaled_df = pd.DataFrame(index=df.index, columns=df.columns).fillna(0)
    
    for date, performance in df.iterrows():
        sorted_performance = performance.sort_values()
        long_stocks = sorted_performance.tail(num_long).index
        short_stocks = sorted_performance.head(num_short).index
        
        # scale the long and short performance scores
        long_sum = np.abs(sorted_performance.loc[long_stocks]).sum()
        short_sum = np.abs(sorted_performance.loc[short_stocks]).sum()
        scaled_df.loc[date, long_stocks] = sorted_performance.loc[long_stocks].div(long_sum) * asset_allocation * ls_total
        scaled_df.loc[date, short_stocks] = sorted_performance.loc[short_stocks].div(short_sum) * (1 - asset_allocation) * ls_total
        
    return scaled_df

# testing
# temp = [-1.55, -2.6, -2.3, -1.87, -3, -1.5, -2, -5, -3.3, -4.3, -0.9]
# df = pd.DataFrame([temp], columns=[f"Column_{i+1}" for i in range(len(temp))])
# scale_scores(df)

## Alphas & Backtesting

In [23]:
class Alphas():
    def __init__(self, df_data):
        self.open = df_data['Open']
        self.high = df_data['High']
        self.low = df_data['Low']
        self.close = df_data['Close']
        self.adjclose = df_data['Adj Close']
        self.volume = df_data['Volume']
        self.returns = returns(df_data['Adj Close'])
        
    def alpha001(self):
        df = -1 * tscorrelation(tsrank(self.close), tsrank(self.volume), 10)
        df = scale_scores(demean(rank(df)))
        return df.replace([-np.inf, np.inf], 0).fillna(value=0)
    
    def alpha002(self):
        df = -self.returns
        df = scale_scores(demean(group_rank(decay_linear(df))))
        return df.replace([-np.inf, np.inf], 0).fillna(value=0)
        
    def alpha003(self):
        df = tsskew(self.returns)
        df = scale_scores(demean(rank(decay_linear(df,2))))
        return df.replace([-np.inf, np.inf], 0).fillna(value=0)

In [24]:
alphas = Alphas(rebalanced_v3)
alpha001 = alphas.alpha001()
alpha002 = alphas.alpha002()
alpha003 = alphas.alpha003()

In [34]:
def get_backtesting_metrics(alpha, rebalanced_v3=rebalanced_v3, STARTING_CAPITAL = 200000):
    
    TRADING_DAYS = 252
    rebalanced_v3.index = pd.to_datetime(rebalanced_v3.index)
    df_close = rebalanced_v3['Close']
    daily_returns_df = df_close.pct_change()

    # shift alpha for the one-day delay in exposure (alphas on t-1 to indicate exposure on t)
    alpha_shifted_df = alpha.shift(1)

    daily_pnl = (alpha_shifted_df * daily_returns_df).sum(axis=1)
    cumulative_pnl = daily_pnl.cumsum() + STARTING_CAPITAL

    daily_log_returns = np.log(cumulative_pnl/cumulative_pnl.shift(1))
    annual_log_return = daily_log_returns.resample('Y').sum()

    daily_turnover = alpha.diff().abs().sum(axis=1) / alpha.abs().sum(axis=1)

    # compile the daily metrics
    daily_metrics = pd.DataFrame({
        'Daily PnL': daily_pnl,
        'Cumulative PnL': cumulative_pnl,
        'Daily Log Return': daily_log_returns,
        'Daily % Turnover': daily_turnover,
        'Daily Win': (daily_pnl > 0).astype(int)
    })

    # calculate the annual metrics
    annual_metrics = pd.DataFrame({
        'Annual PnL': daily_metrics['Daily PnL'].resample('Y').sum(),
        'Cumulative PnL': daily_metrics['Cumulative PnL'].resample('Y').last(),
        'Log Return': annual_log_return,
        '% Turnover': daily_metrics['Daily % Turnover'].resample('Y').mean() * 100,
        '% Win': daily_metrics['Daily Win'].resample('Y').mean() * 100,
    })
    
    def calculate_annual_max_drawdown(cumulative_pnl):
        drawdown = cumulative_pnl.div(cumulative_pnl.cummax()).sub(1)
        return drawdown.resample('Y').min()

    daily_return_std_dev = daily_log_returns.std()
    information_ratio = annual_log_return / (daily_return_std_dev * np.sqrt(TRADING_DAYS))
    annual_metrics['Information Ratio'] = information_ratio
    sharpe_ratio = (annual_log_return-0.02) / (daily_return_std_dev * np.sqrt(TRADING_DAYS))
    annual_metrics['Sharpe Ratio'] = sharpe_ratio
    annual_metrics['% Max Drawdown'] = -calculate_annual_max_drawdown(daily_metrics['Cumulative PnL']) * 100

    annual_metrics.index = annual_metrics.index.year
    return annual_metrics, daily_log_returns

In [26]:
alpha_returns1 = np.array(get_backtesting_metrics(alpha001)[1].dropna())
alpha_returns2 = np.array(get_backtesting_metrics(alpha002)[1].dropna())
alpha_returns3 = np.array(get_backtesting_metrics(alpha003)[1].dropna())

In [35]:
get_backtesting_metrics(alpha002)[0]

,Annual PnL,Cumulative PnL,Log Return,% Turnover,% Win,Information Ratio,Sharpe Ratio,% Max Drawdown
Date,,,,,,,,
2016,8480.598692,208480.598692,0.041529,39.924075,55.778894,0.817154,0.423616,3.562978
2017,12715.962905,221196.561598,0.059206,40.487661,54.980080,1.164984,0.771447,1.654064
2018,10551.145127,231747.706725,0.046598,40.854488,53.784861,0.916895,0.523357,2.716685
2019,3677.318426,235425.025151,0.015743,39.789386,55.158730,0.309777,-0.083761,4.321632
2020,45489.572394,280914.597545,0.176658,42.361523,53.754941,3.476082,3.082545,12.993006
2021,-604.594645,280310.002901,-0.002155,40.077653,51.587302,-0.042395,-0.435933,3.122352
2022,-1065.435707,279244.567194,-0.003808,39.942988,50.996016,-0.074933,-0.468470,7.616566
2023,1879.623487,281124.190681,0.006709,39.298550,50.847458,0.132003,-0.261534,4.306771


## Alphas Combination

In [30]:
# calculate expected returns and covariance matrix for alphas
alpha_returns = np.column_stack((alpha_returns1, alpha_returns2, alpha_returns3))
expected_alpha_returns = alpha_returns.mean(axis=0)
alpha_cov_matrix = np.cov(alpha_returns.T)

# define the objective function for optimization (negative Sharpe ratio)
def neg_sharpe(weights, mean_returns, cov_matrix, risk_free_rate=0.02):
    TRADING_DAYS = 252
    p_returns = np.dot(weights, mean_returns)*TRADING_DAYS
    p_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(TRADING_DAYS)
    return -(p_returns - risk_free_rate) / p_vol

# constraints and bounds for the optimization
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # Weights must sum to 1
bounds = [(0, 1) for _ in range(len(expected_alpha_returns))]  # Weights between 0 and 1

# initial guess (equal weighting)
init_guess = np.ones(len(expected_alpha_returns))/len(expected_alpha_returns)

# minimize the negative Sharpe = maximize the Sharpe
opt_result = minimize(neg_sharpe, init_guess, args=(expected_alpha_returns, alpha_cov_matrix), 
                      method='SLSQP', bounds=bounds, constraints=constraints)

TRADING_DAYS = 252
optimal_weights, sharpe = opt_result.x, -opt_result.fun
optimal_returns = np.dot(optimal_weights, expected_alpha_returns) * TRADING_DAYS
optimal_volatility = np.sqrt(np.dot(optimal_weights.T, np.dot(alpha_cov_matrix, optimal_weights))) * np.sqrt(TRADING_DAYS) 

print("Optimal annualized returns:", optimal_returns)
print("Optimal annualized volatility:", optimal_volatility)
print("Optimal weights for alpha signals:", optimal_weights)
print("Calculated Sharpe ratio:", sharpe)

Optimal annualized returns: 0.03869613618628342
Optimal annualized volatility: 0.02688316331350385
Optimal weights for alpha signals: [0.         0.38444146 0.61555854]
Calculated Sharpe ratio: 0.6954589371888407


## S&P500 Sharpe

In [31]:
start_date = '2016-03-21'
end_date = '2023-12-02'
index = yf.download('^GSPC', start=start_date, end=end_date)
index

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-03-21,2047.880005,2053.909912,2043.140015,2051.600098,2051.600098,3376600000
2016-03-22,2048.639893,2056.600098,2040.569946,2049.800049,2049.800049,3418460000
2016-03-23,2048.550049,2048.550049,2034.859985,2036.709961,2036.709961,3639510000
2016-03-24,2032.479980,2036.040039,2022.489990,2035.939941,2035.939941,3407720000
2016-03-28,2037.890015,2042.670044,2031.959961,2037.050049,2037.050049,2809090000
...,...,...,...,...,...,...
2023-11-27,4554.859863,4560.520020,4546.319824,4550.430176,4550.430176,3403990000
2023-11-28,4545.549805,4568.140137,4540.509766,4554.890137,4554.890137,3586240000
2023-11-29,4571.839844,4587.640137,4547.149902,4550.580078,4550.580078,4418760000


In [33]:
index_returns = np.log(index['Adj Close']/index['Adj Close'].shift(1))
index_stddev = index_returns.std()
print("S&P500 Sharpe:", ((index_returns.mean()*252 - 0.02)/(index_stddev.mean()*np.sqrt(252))))

S&P500 Sharpe: 0.45209324014443614
